# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-03 07:37:42] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=36512, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=296484272, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_

[2025-05-03 07:37:53] Attention backend not set. Use fa3 backend by default.
[2025-05-03 07:37:53] Init torch distributed begin.


[2025-05-03 07:37:53] Init torch distributed ends. mem usage=0.00 GB
[2025-05-03 07:37:53] Load weight begin. avail mem=35.23 GB


[2025-05-03 07:37:53] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-03 07:37:54] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-05-03 07:37:57] KV Cache is allocated. #tokens: 20480, K size: 0.55 GB, V size: 0.55 GB
[2025-05-03 07:37:57] Memory pool end. avail mem=19.22 GB


[2025-05-03 07:37:57] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-03 07:37:58] INFO:     Started server process [2140603]
[2025-05-03 07:37:58] INFO:     Waiting for application startup.
[2025-05-03 07:37:58] INFO:     Application startup complete.
[2025-05-03 07:37:58] INFO:     Uvicorn running on http://0.0.0.0:36512 (Press CTRL+C to quit)


[2025-05-03 07:37:58] INFO:     127.0.0.1:45422 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-03 07:37:59] INFO:     127.0.0.1:45430 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-03 07:37:59] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-03 07:38:06] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-05-03 07:38:06] INFO:     127.0.0.1:45442 - "POST /generate HTTP/1.1" 200 OK
[2025-05-03 07:38:06] The server is fired up and ready to roll!


[2025-05-03 07:38:07] Decode batch. #running-req: 1, #token: 53, token usage: 0.00, gen throughput (token/s): 3.85, #queue-req: 0


[2025-05-03 07:38:08] Decode batch. #running-req: 1, #token: 93, token usage: 0.00, gen throughput (token/s): 106.79, #queue-req: 0


[2025-05-03 07:38:08] Decode batch. #running-req: 1, #token: 133, token usage: 0.01, gen throughput (token/s): 106.09, #queue-req: 0


[2025-05-03 07:38:09] Decode batch. #running-req: 1, #token: 173, token usage: 0.01, gen throughput (token/s): 107.20, #queue-req: 0


...

[2025-05-03 07:38:25] Decode batch. #running-req: 1, #token: 1893, token usage: 0.09, gen throughput (token/s): 104.20, #queue-req: 0


[2025-05-03 07:38:26] Decode batch. #running-req: 1, #token: 1933, token usage: 0.09, gen throughput (token/s): 103.58, #queue-req: 0


[2025-05-03 07:38:26] Decode batch. #running-req: 1, #token: 1973, token usage: 0.10, gen throughput (token/s): 101.88, #queue-req: 0


[2025-05-03 07:38:26] Decode batch. #running-req: 1, #token: 2013, token usage: 0.10, gen throughput (token/s): 105.21, #queue-req: 0


[2025-05-03 07:38:27] Decode batch. #running-req: 1, #token: 2053, token usage: 0.10, gen throughput (token/s): 106.26, #queue-req: 0
[2025-05-03 07:38:27] INFO:     127.0.0.1:45454 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-03 07:38:27] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 07:38:27] Decode batch. #running-req: 1, #token: 45, token usage: 0.00, gen throughput (token/s): 74.88, #queue-req: 0


[2025-05-03 07:38:28] Decode batch. #running-req: 1, #token: 85, token usage: 0.00, gen throughput (token/s): 108.68, #queue-req: 0


[2025-05-03 07:38:28] Decode batch. #running-req: 1, #token: 125, token usage: 0.01, gen throughput (token/s): 108.91, #queue-req: 0


[2025-05-03 07:38:28] Decode batch. #running-req: 1, #token: 165, token usage: 0.01, gen throughput (token/s): 109.51, #queue-req: 0


...

[2025-05-03 07:38:46] Decode batch. #running-req: 1, #token: 1925, token usage: 0.09, gen throughput (token/s): 66.80, #queue-req: 0


[2025-05-03 07:38:46] Decode batch. #running-req: 1, #token: 1965, token usage: 0.10, gen throughput (token/s): 64.89, #queue-req: 0


[2025-05-03 07:38:47] Decode batch. #running-req: 1, #token: 2005, token usage: 0.10, gen throughput (token/s): 65.35, #queue-req: 0


[2025-05-03 07:38:47] Decode batch. #running-req: 1, #token: 2045, token usage: 0.10, gen throughput (token/s): 64.58, #queue-req: 0


[2025-05-03 07:38:48] INFO:     127.0.0.1:45454 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-03 07:38:48] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 07:38:48] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, gen throughput (token/s): 58.92, #queue-req: 0


[2025-05-03 07:38:49] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, gen throughput (token/s): 65.28, #queue-req: 0


[2025-05-03 07:38:49] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, gen throughput (token/s): 66.12, #queue-req: 0


[2025-05-03 07:38:50] INFO:     127.0.0.1:45454 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-03 07:38:50] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 07:38:50] Decode batch. #running-req: 1, #token: 34, token usage: 0.00, gen throughput (token/s): 61.96, #queue-req: 0


[2025-05-03 07:38:51] Decode batch. #running-req: 1, #token: 74, token usage: 0.00, gen throughput (token/s): 65.44, #queue-req: 0


[2025-05-03 07:38:51] Decode batch. #running-req: 1, #token: 114, token usage: 0.01, gen throughput (token/s): 65.82, #queue-req: 0


[2025-05-03 07:38:52] Decode batch. #running-req: 1, #token: 154, token usage: 0.01, gen throughput (token/s): 65.41, #queue-req: 0


[2025-05-03 07:38:52] Decode batch. #running-req: 1, #token: 194, token usage: 0.01, gen throughput (token/s): 65.26, #queue-req: 0


[2025-05-03 07:38:53] Decode batch. #running-req: 1, #token: 234, token usage: 0.01, gen throughput (token/s): 74.49, #queue-req: 0


[2025-05-03 07:38:53] Decode batch. #running-req: 1, #token: 274, token usage: 0.01, gen throughput (token/s): 109.33, #queue-req: 0


[2025-05-03 07:38:54] Decode batch. #running-req: 1, #token: 314, token usage: 0.02, gen throughput (token/s): 110.05, #queue-req: 0


[2025-05-03 07:38:54] Decode batch. #running-req: 1, #token: 354, token usage: 0.02, gen throughput (token/s): 109.81, #queue-req: 0
[2025-05-03 07:38:54] INFO:     127.0.0.1:45454 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-03 07:38:55] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 07:38:55] Decode batch. #running-req: 1, #token: 496, token usage: 0.02, gen throughput (token/s): 53.43, #queue-req: 0


[2025-05-03 07:38:55] Decode batch. #running-req: 1, #token: 536, token usage: 0.03, gen throughput (token/s): 88.52, #queue-req: 0


[2025-05-03 07:38:56] Decode batch. #running-req: 1, #token: 576, token usage: 0.03, gen throughput (token/s): 107.38, #queue-req: 0


[2025-05-03 07:38:56] Decode batch. #running-req: 1, #token: 616, token usage: 0.03, gen throughput (token/s): 70.86, #queue-req: 0


...

[2025-05-03 07:39:00] Decode batch. #running-req: 1, #token: 896, token usage: 0.04, gen throughput (token/s): 106.04, #queue-req: 0


[2025-05-03 07:39:00] Decode batch. #running-req: 1, #token: 936, token usage: 0.05, gen throughput (token/s): 105.71, #queue-req: 0


[2025-05-03 07:39:00] Decode batch. #running-req: 1, #token: 976, token usage: 0.05, gen throughput (token/s): 105.97, #queue-req: 0


[2025-05-03 07:39:01] Decode batch. #running-req: 1, #token: 1016, token usage: 0.05, gen throughput (token/s): 105.47, #queue-req: 0


[2025-05-03 07:39:01] INFO:     127.0.0.1:45454 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-03 07:39:02] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-03 07:39:02] Decode batch. #running-req: 1, #token: 24, token usage: 0.00, gen throughput (token/s): 39.45, #queue-req: 0


[2025-05-03 07:39:02] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, gen throughput (token/s): 107.03, #queue-req: 0


[2025-05-03 07:39:03] Decode batch. #running-req: 1, #token: 104, token usage: 0.01, gen throughput (token/s): 108.27, #queue-req: 0


[2025-05-03 07:39:03] Decode batch. #running-req: 1, #token: 144, token usage: 0.01, gen throughput (token/s): 108.36, #queue-req: 0


[2025-05-03 07:39:03] Decode batch. #running-req: 1, #token: 184, token usage: 0.01, gen throughput (token/s): 108.19, #queue-req: 0


...

[2025-05-03 07:39:20] Decode batch. #running-req: 1, #token: 1904, token usage: 0.09, gen throughput (token/s): 108.65, #queue-req: 0


[2025-05-03 07:39:20] Decode batch. #running-req: 1, #token: 1944, token usage: 0.09, gen throughput (token/s): 97.08, #queue-req: 0


[2025-05-03 07:39:20] Decode batch. #running-req: 1, #token: 1984, token usage: 0.10, gen throughput (token/s): 106.58, #queue-req: 0


[2025-05-03 07:39:21] Decode batch. #running-req: 1, #token: 2024, token usage: 0.10, gen throughput (token/s): 101.26, #queue-req: 0


[2025-05-03 07:39:21] Decode batch. #running-req: 1, #token: 2064, token usage: 0.10, gen throughput (token/s): 105.10, #queue-req: 0
[2025-05-03 07:39:21] INFO:     127.0.0.1:50038 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not entirely sure about all the details, but I\'ll try to think it through.\n\nFirst, I know that the capital of France is Paris. That\'s pretty much a given, right? But I should double-check that. Maybe I can recall any other capitals I know. London is the capital of the UK, Rome is Italy, and maybe Tokyo is Japan\'s. Yeah, Paris seems correct for France.\n\nNow, moving on to the population. I think Paris is a very large city, but I\'m not sure of the exact number. I remember it\'s over 3 million, but I\'m not certain. Maybe around 3.5 million? I should probably look that up, but since I can\'t right now, I\'ll go with 3,500,000 as an estimate.\n\nNext, the area. Par

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-03 07:39:21] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 07:39:22] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 101.43, #queue-req: 0


[2025-05-03 07:39:22] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 110.33, #queue-req: 0


[2025-05-03 07:39:22] Decode batch. #running-req: 1, #token: 121, token usage: 0.01, gen throughput (token/s): 109.52, #queue-req: 0


[2025-05-03 07:39:23] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, gen throughput (token/s): 108.87, #queue-req: 0


...

[2025-05-03 07:39:27] Decode batch. #running-req: 1, #token: 641, token usage: 0.03, gen throughput (token/s): 107.21, #queue-req: 0


[2025-05-03 07:39:27] Decode batch. #running-req: 1, #token: 681, token usage: 0.03, gen throughput (token/s): 106.82, #queue-req: 0


[2025-05-03 07:39:28] Decode batch. #running-req: 1, #token: 721, token usage: 0.04, gen throughput (token/s): 106.44, #queue-req: 0


[2025-05-03 07:39:28] Decode batch. #running-req: 1, #token: 761, token usage: 0.04, gen throughput (token/s): 105.72, #queue-req: 0


[2025-05-03 07:39:29] INFO:     127.0.0.1:57318 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-03 07:39:29] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-03 07:39:29] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 07:39:29] Decode batch. #running-req: 3, #token: 26, token usage: 0.00, gen throughput (token/s): 83.87, #queue-req: 0


[2025-05-03 07:39:29] Decode batch. #running-req: 3, #token: 146, token usage: 0.01, gen throughput (token/s): 303.87, #queue-req: 0


[2025-05-03 07:39:30] Decode batch. #running-req: 3, #token: 266, token usage: 0.01, gen throughput (token/s): 306.28, #queue-req: 0


[2025-05-03 07:39:30] Decode batch. #running-req: 3, #token: 386, token usage: 0.02, gen throughput (token/s): 306.66, #queue-req: 0


[2025-05-03 07:39:30] Decode batch. #running-req: 3, #token: 506, token usage: 0.02, gen throughput (token/s): 308.47, #queue-req: 0


[2025-05-03 07:39:31] INFO:     127.0.0.1:51510 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other major city in France being referred to as the capital. So, I'm pretty confident that Paris is correct.\n</think>Paris is the capital of F

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-03 07:39:31] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-03 07:39:31] Decode batch. #running-req: 1, #token: 10, token usage: 0.00, gen throughput (token/s): 267.73, #queue-req: 0


[2025-05-03 07:39:31] Decode batch. #running-req: 1, #token: 50, token usage: 0.00, gen throughput (token/s): 108.50, #queue-req: 0


[2025-05-03 07:39:32] Decode batch. #running-req: 1, #token: 90, token usage: 0.00, gen throughput (token/s): 108.34, #queue-req: 0


[2025-05-03 07:39:32] Decode batch. #running-req: 1, #token: 130, token usage: 0.01, gen throughput (token/s): 108.43, #queue-req: 0


[2025-05-03 07:39:32] Decode batch. #running-req: 1, #token: 170, token usage: 0.01, gen throughput (token/s): 108.49, #queue-req: 0


...

[2025-05-03 07:39:49] Decode batch. #running-req: 1, #token: 1890, token usage: 0.09, gen throughput (token/s): 106.89, #queue-req: 0


[2025-05-03 07:39:49] Decode batch. #running-req: 1, #token: 1930, token usage: 0.09, gen throughput (token/s): 106.83, #queue-req: 0


[2025-05-03 07:39:50] Decode batch. #running-req: 1, #token: 1970, token usage: 0.10, gen throughput (token/s): 107.05, #queue-req: 0


[2025-05-03 07:39:50] Decode batch. #running-req: 1, #token: 2010, token usage: 0.10, gen throughput (token/s): 107.27, #queue-req: 0


[2025-05-03 07:39:50] Decode batch. #running-req: 1, #token: 2050, token usage: 0.10, gen throughput (token/s): 106.88, #queue-req: 0
[2025-05-03 07:39:50] INFO:     127.0.0.1:51526 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-03 07:39:51] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-03 07:39:51] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, gen throughput (token/s): 101.33, #queue-req: 0


[2025-05-03 07:39:51] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, gen throughput (token/s): 108.24, #queue-req: 0


[2025-05-03 07:39:52] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, gen throughput (token/s): 108.25, #queue-req: 0


[2025-05-03 07:39:52] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, gen throughput (token/s): 108.19, #queue-req: 0


...

[2025-05-03 07:40:00] Decode batch. #running-req: 1, #token: 1016, token usage: 0.05, gen throughput (token/s): 106.65, #queue-req: 0


[2025-05-03 07:40:00] Decode batch. #running-req: 1, #token: 1056, token usage: 0.05, gen throughput (token/s): 106.27, #queue-req: 0


[2025-05-03 07:40:01] Decode batch. #running-req: 1, #token: 1096, token usage: 0.05, gen throughput (token/s): 105.99, #queue-req: 0


[2025-05-03 07:40:01] Decode batch. #running-req: 1, #token: 1136, token usage: 0.06, gen throughput (token/s): 104.86, #queue-req: 0


[2025-05-03 07:40:01] INFO:     127.0.0.1:48724 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

[2025-05-03 07:40:01] Child process unexpectedly failed with an exit code 9. pid=2141194


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.28s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.20s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.21s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text:  the capital of France is Paris. Paris is known as the "City of Light" because of its many bridges, especially the Eiffel Tower Bridge. It is also famous for its cuisine, such as baguette, croissant, and artichoke. Paris has a rich history as the capital of France. Today, it is a major center of culture and business in Europe. There are many museums, such as the Louvre and the Musée d'Orsay, and many operas, such as the Opéra Garnier. Paris also has a vibrant nightlife, with popular clubs and bars.

Okay, so I need
Prompt: Give me the information of the capital of Germany.
Generated text: 
Okay, so I need to figure out the capital of Germany. Hmm, I remember that Germany is a country in Europe, but I'm not exactly sure which city is its capital. I think it's Berlin, but wait, isn't there a city called Berlin? Maybe that's the capital. Let me try to recall more details. I believe Berlin is the capital, and it's al

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its economic status, cultural significance, and geographical location.9-10 sentences.

Certainly! Here's a concise overview of London:

**London** is one of the most significant global cities, renowned for its economic prominence, cultural richness, and strategic location. **Economically**, London is a powerhouse, home to an estimated GDP exceeding $6 trillion, making it the world's most valuable city. The financial sector is a cornerstone, hosting the headquarters of nearly all the world's major banks and investment banks. The City alone, with its Stock Exchange, employs over 400,000 people, contributing heavily to the
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, economic significance, and cultural importance.

2. For each of the following statements, determine whether it's true or false. If the statement is false, provide a reason.

Stateme

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Okay, I need to provide a JSON object containing information about the capital of France. Let me start by identifying the key details. The capital is Paris, so that's the first piece. Next, I should note the country, which is France. The current population is approximately 2.2 million as of 2023, but I'll check if that's up to date. The official language is French, so that's next. For the languages spoken, French is the main one, but many people also speak other languages like English, Spanish, German, Italian, and Arabic, so I'll list those. I should include the area, which is about 10.55 square kilometers. The government type is a semi-presidential republic, so that's accurate. Paris is located in Île-de-France, and its time zone is UTC+1. I'll structure all this information into a JSON format, making sure each field is properly named and the

In [19]:
llm.shutdown()